In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
from scipy.io import loadmat
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from  IPython import display
from matplotlib import pyplot as plt
from google.colab import files
import shutil
import os

print(tf.__version__)

from google.colab import drive
drive.mount('/content/drive')

2.9.2
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import keras


In [ ]:
!cp /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/process_image.py .

In [ ]:
!python /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/process_image.py

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:02<00:00, 41.92it/s]
100% 100/100 [00:02<00:00, 42.38it/s]
 92% 739/800 [13:51<01:10,  1.15s/it]

In [ ]:
# !zip -r /content/new_data/kvasir_segmentation_dataset/valid.zip /content/new_data/kvasir_segmentation_dataset/

In [ ]:
!cp /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/unet.py .
!python /content/unet.py

In [ ]:
from google.colab import files
files.view('/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/run.py')

<IPython.core.display.Javascript object>

In [ ]:
!cp /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/data_generator.py .
!python /content/data_generator.py
!cp /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/metrics.py .
!python /content/metrics.py

In [ ]:
!python data_generator.py

In [ ]:
!cp /content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/run.py .


In [ ]:
!python run.py

2022-10-12 18:44:36.914805: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
run.py:86: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  callbacks=callbacks)
Epoch 1/20
 24/250 [=>............................] - ETA: 44:00 - loss: 0.7371 - recall: 0.5553 - precision: 0.2418 - dice_coef: 0.2629 - mean_io_u: 0.4206^C


In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
# from resunet import ResUnet
# from m_resunet import ResUnetPlusPlus
from metrics import dice_coef, dice_loss

In [ ]:
file_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/files/"
model_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/files/unet_try1.h5"

## Create files folder
try:
    os.mkdir("/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/files")
except:
    pass

train_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/new_data/kvasir_segmentation_dataset/train/"
valid_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/new_data/kvasir_segmentation_dataset/valid/"


In [ ]:
## Training
train_image_paths = glob(os.path.join(train_path, "images", "*"))
train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
train_image_paths.sort()
train_mask_paths.sort()

train_image_paths = train_image_paths[:2000]
train_mask_paths = train_mask_paths[:2000]

## Validation
valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
valid_image_paths.sort()
valid_mask_paths.sort()

In [ ]:
## Parameters
image_size = 256
batch_size = 64
lr = 1e-4
epochs = 200

train_steps = len(train_image_paths)//batch_size
valid_steps = len(valid_image_paths)//batch_size

## Generator
train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)


In [ ]:
len(train_gen)



32

In [ ]:
arch = Unet(input_size=image_size)
model = arch.build_model()

In [ ]:
model.summary()

In [ ]:
optimizer = Nadam(lr)
metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2)]
model.compile(loss=dice_loss, optimizer=optimizer, metrics=metrics)

csv_logger = CSVLogger(f"{file_path}unet_{batch_size}.csv", append=False)
checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]


In [ ]:
model.fit_generator(train_gen,
            validation_data=valid_gen,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            epochs=epochs,
            callbacks=callbacks)


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence


def parse_image(img_path, image_size):
    image_rgb = cv2.imread(img_path, 1)
    h, w, _ = image_rgb.shape
    if (h == image_size) and (w == image_size):
        pass
    else:
        image_rgb = cv2.resize(image_rgb, (image_size, image_size))
    image_rgb = image_rgb/255.0
    return image_rgb

def parse_mask(mask_path, image_size):
    mask = cv2.imread(mask_path, -1)
    h, w = mask.shape
    if (h == image_size) and (w == image_size):
        pass
    else:
        mask = cv2.resize(mask, (image_size, image_size))
    mask = np.expand_dims(mask, -1)
    mask = mask/255.0

    return mask


In [ ]:
train_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/new_data/kvasir_segmentation_dataset/train/"
test_path = "/content/drive/MyDrive/dip_project/ResUNetPlusPlus-master/new_data/kvasir_segmentation_dataset/test/"


In [ ]:
len(train_path)

101

In [ ]:
import tensorflow as tf
import os
import random
import numpy as np

from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

seed = 42
np.random.seed = seed

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

train_ids_img = next(os.walk(train_path+'/images/'))[2]
train_ids_mask = next(os.walk(train_path+'/masks/'))[2]

test_ids_img = next(os.walk(test_path+'/images/'))[2]
test_ids_mask = next(os.walk(test_path+'/masks/'))[2]

X_train = np.zeros((len(train_ids_img), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids_mask), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

print('Resizing training images and masks')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Resizing training images and masks


In [ ]:
len(test_ids_img)


100

In [ ]:
print('Resizing training images and masks')

for n, id_ in tqdm(enumerate(train_ids_img), total=len(train_ids_img)):
    path = train_path
    img = imread(path + '/images/' + id_ )[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img  #Fill empty X_train with values from img

for n, id_ in tqdm(enumerate(train_ids_mask), total=len(train_ids_mask)):
    path = train_path
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    mask_ = imread(path + '/masks/' + id_ )
    mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
    mask = np.maximum(mask, mask_)
    Y_train[n] = mask